In [ ]:
# %pip install tensorflow-datasets

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import ReduceLROnPlateau

In [ ]:
from utils import train_cache, val_ds, show_results, dataset_object
from model import make_model

In [ ]:
model = make_model(
    lr_shape=(None, None, 3),
    num_filters=16,
    num_of_residual_blocks_a=8,
    num_of_residual_blocks_b=2,
)

In [ ]:
# model = tf.keras.models.load_model("../models/edsr-12.keras")

In [ ]:
optim_edsr = keras.optimizers.Adam(5e-04, beta_1=0.5)
tb_callback = keras.callbacks.TensorBoard("./logs", update_freq=1, write_images=True)

learning_rate_reduction = ReduceLROnPlateau(
    monitor="val_loss", factor=0.5, patience=10, min_lr=1e-8, verbose=1
)

# Compiling model with loss as mean absolute error(L1 Loss) and metric as psnr
model.compile(optimizer=optim_edsr, loss="mae", metrics=["accuracy"])

In [ ]:
train_ds_0 = dataset_object(train_cache, training=True, hr_crop_size=32)
train_ds_1 = dataset_object(train_cache, training=True, hr_crop_size=64)
train_ds_2 = dataset_object(train_cache, training=True, hr_crop_size=128)
train_ds_3 = dataset_object(train_cache, training=True, hr_crop_size=256)

train_ds = [train_ds_0, train_ds_1, train_ds_2, train_ds_3]
train_epochs = [50, 50, 25, 10]

In [ ]:
# Training for more epochs will improve results
show_results(model, num_images=3, size=256)

for dataset, te in zip(train_ds, train_epochs):
    print(f"==================== Training ds changed for epochs ====================")
    model.fit(
        dataset,
        epochs=te,
        steps_per_epoch=512,
        validation_data=val_ds,
        callbacks=[tb_callback, learning_rate_reduction],
    )

    show_results(model, num_images=3, size=256)

In [ ]:
tf.keras.Model.save(model, "../models/edsr-13.keras")

In [ ]:
model.save_weights("../models/edsr-13.weights.h5")